In [4]:
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import sequence
import numpy as np

In [2]:
vocab_size = 5000
(x_train, y_train), (x_test, y_test) = imdb.load_data(path = 'C:/Users/xz328e/Documents/Journey_To_Transformers/imdb.npz', num_words=vocab_size)

# Getting all the words from word_index dictionary
word_idx = imdb.get_word_index(path = 'C:/Users/xz328e/Documents/Journey_To_Transformers/imdb_word_index.json')

# Originally the index number of a value and not a key,
# hence converting the index as key and the words as values
word_idx = {i: word for word, i in word_idx.items()}

# again printing the review
print([word_idx[i] for i in x_train[0]])

['the', 'as', 'you', 'with', 'out', 'themselves', 'powerful', 'lets', 'loves', 'their', 'becomes', 'reaching', 'had', 'journalist', 'of', 'lot', 'from', 'anyone', 'to', 'have', 'after', 'out', 'atmosphere', 'never', 'more', 'room', 'and', 'it', 'so', 'heart', 'shows', 'to', 'years', 'of', 'every', 'never', 'going', 'and', 'help', 'moments', 'or', 'of', 'every', 'chest', 'visual', 'movie', 'except', 'her', 'was', 'several', 'of', 'enough', 'more', 'with', 'is', 'now', 'current', 'film', 'as', 'you', 'of', 'mine', 'potentially', 'unfortunately', 'of', 'you', 'than', 'him', 'that', 'with', 'out', 'themselves', 'her', 'get', 'for', 'was', 'camp', 'of', 'you', 'movie', 'sometimes', 'movie', 'that', 'with', 'scary', 'but', 'and', 'to', 'story', 'wonderful', 'that', 'in', 'seeing', 'in', 'character', 'to', 'of', '70s', 'and', 'with', 'heart', 'had', 'shadows', 'they', 'of', 'here', 'that', 'with', 'her', 'serious', 'to', 'have', 'does', 'when', 'from', 'why', 'what', 'have', 'critics', 'they'

In [3]:
# Get the minimum and the maximum length of reviews
print("Max length of a review:: ", len(max((x_train+x_test), key=len)))
print("Min length of a review:: ", len(min((x_train+x_test), key=len)))

Max length of a review::  2697
Min length of a review::  70


In [5]:
# Keeping a fixed length of all reviews to max 400 words
max_words = 400

x_train = sequence.pad_sequences(x_train, maxlen=max_words)
x_test = sequence.pad_sequences(x_test, maxlen=max_words)

x_valid, y_valid = x_train[:64], y_train[:64]
x_train_, y_train_ = x_train[64:], y_train[64:]

In [6]:
# fixing every word's embedding size to be 32
embd_len = 32

# Creating a RNN model
RNN_model = Sequential(name="Simple_RNN")
RNN_model.add(Embedding(vocab_size,
                        embd_len,
                        input_length=max_words))

# In case of a stacked(more than one layer of RNN)
# use return_sequences=True
RNN_model.add(SimpleRNN(128,
                        activation='tanh',
                        return_sequences=False))
RNN_model.add(Dense(1, activation='sigmoid'))

# printing model summary
print(RNN_model.summary())

# Compiling model
RNN_model.compile(
    loss="binary_crossentropy",
    optimizer='adam',
    metrics=['accuracy']
)

# Training the model
history = RNN_model.fit(x_train_, y_train_,
                        batch_size=64,
                        epochs=5,
                        verbose=1,
                        validation_data=(x_valid, y_valid))

# Printing model score on test data
print()
print("Simple_RNN Score---> ", RNN_model.evaluate(x_test, y_test, verbose=0))

Model: "Simple_RNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 400, 32)           160000    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 128)               20608     
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 180,737
Trainable params: 180,737
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
390/390 [==============================] - 44s 112ms/step - loss: 0.6868 - accuracy: 0.5403 - val_loss: 0.6972 - val_accuracy: 0.4688
Epoch 2/5
390/390 [==============================] - 44s 114ms/step - loss: 0.6471 - accuracy: 0.6262 - val_loss: 0.6428 - val_accuracy: 0.6094
Epoch 3/5
390/390 [==============================] - 48s 124ms/step - loss: 0.4818 - ac

The vanilla form of RNN gave us a Test Accuracy of 64.95%. Limitations of Simple RNN are it is unable to handle long sentences well because of its vanishing gradient problems.

In [7]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Example sentence
text = "Bob single-handedly fought the enemy and died for his country. For his contributions, brave ______."

# Step 1: Preprocess the text
sentences = [text.lower()]  # Lowercase for consistency
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

# Tokenize the sentence
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1  # Vocabulary size (+1 for padding token)
sequences = tokenizer.texts_to_sequences(sentences)

# Create input (X) and output (y) pairs for training
sequence = sequences[0]
X, y = [], []
for i in range(1, len(sequence)):
    X.append(sequence[:i])  # Input sequence up to the current word
    y.append(sequence[i])   # The next word

# Pad sequences to make them of equal length
max_len = max(len(seq) for seq in X)
X = pad_sequences(X, maxlen=max_len, padding='pre')
y = np.array(y)

# Step 2: Define the RNN model
embedding_dim = 100
hidden_dim = 128

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len),
    SimpleRNN(hidden_dim, return_sequences=False),  # Simple RNN layer
    Dense(vocab_size, activation='softmax')         # Output layer
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Step 3: Train the model
model.fit(X, y, epochs=100, verbose=1)

# Step 4: Predict the next word
def predict_next_word(input_text, model, tokenizer, max_len):
    # Tokenize and pad the input text
    sequence = tokenizer.texts_to_sequences([input_text])[0]
    sequence = pad_sequences([sequence], maxlen=max_len, padding='pre')
    
    # Predict the next word
    predicted_index = np.argmax(model.predict(sequence), axis=-1)[0]
    for word, index in tokenizer.word_index.items():
        if index == predicted_index:
            return word
    return None

# Input for prediction
input_text = "bob single-handedly fought the enemy and died for his country. for his contributions, brave"
predicted_word = predict_next_word(input_text, model, tokenizer, max_len)
print(f"Predicted next word: {predicted_word}")


Epoch 1/100
1/1 [==============================] - 1s 1s/step - loss: 2.6557 - accuracy: 0.0714
Epoch 2/100
1/1 [==============================] - 0s 8ms/step - loss: 2.5232 - accuracy: 0.2857
Epoch 3/100
1/1 [==============================] - 0s 9ms/step - loss: 2.3966 - accuracy: 0.4286
Epoch 4/100
1/1 [==============================] - 0s 7ms/step - loss: 2.2689 - accuracy: 0.5000
Epoch 5/100
1/1 [==============================] - 0s 9ms/step - loss: 2.1380 - accuracy: 0.5000
Epoch 6/100
1/1 [==============================] - 0s 10ms/step - loss: 2.0046 - accuracy: 0.5714
Epoch 7/100
1/1 [==============================] - 0s 7ms/step - loss: 1.8710 - accuracy: 0.5714
Epoch 8/100
1/1 [==============================] - 0s 8ms/step - loss: 1.7413 - accuracy: 0.5714
Epoch 9/100
1/1 [==============================] - 0s 7ms/step - loss: 1.6191 - accuracy: 0.5714
Epoch 10/100
1/1 [==============================] - 0s 7ms/step - loss: 1.5033 - accuracy: 0.5714
Epoch 11/100
1/1 [===========